<a href="https://colab.research.google.com/github/Xinxinatg/CSRNet-pytorch/blob/master/make_dataset_and_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!wget --header="Host: doc-0o-64-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-AU,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,en-GB;q=0.6,en-US;q=0.5" --header="Cookie: AUTH_9hg3ee1ib7ptld264s2flqgrf9hg9jro_nonce=7p5l7uis53e3c" --header="Connection: keep-alive" "https://doc-0o-64-docs.googleusercontent.com/docs/securesc/0chsofepnd39clr7fg0d2tu89snbaa74/f2bco2an79m1lb32hb7c0bg56ebngt8d/1603958700000/07527692003091305349/07527692003091305349/128T12A_F_e_K_PBDog4RqsB19Qjq0kx-?e=download&authuser=0&nonce=7p5l7uis53e3c&user=07527692003091305349&hash=4tslfbim8n9bq7eup8iok00c3vd6cooq" -c -O 'VisDrone2020-CC.zip'

In [5]:
#!unzip -q /content/VisDrone2020-CC.zip

In [6]:
#!git clone https://github.com/Xinxinatg/CSRNet-pytorch.git

In [7]:
#import h5py
#import scipy.io as io
#import sys
#        sys.path.extend(['/content/CSRNet-pytorch'])
 #       import PIL.Image as Image
  #      import numpy as np
   #     import os
    #    import glob
     #   from matplotlib import pyplot as plt
      #  from scipy.ndimage.filters import gaussian_filter 
       # import scipy
        #import json
        #from matplotlib import cm as CM
        #from image import *
        #from model import CSRNet
        #import torch
%matplotlib inline
import json
import h5py
import scipy.io as io
import sys
sys.path.extend(['/content/CSRNet_pytorch'])
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch

In [8]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print gt.shape
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(zip(np.nonzero(gt)[1], np.nonzero(gt)[0]))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print 'generate density...'
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print 'done.'
    return density

In [9]:
#set the root to the visdrone cc dataset
root = '/content/VisDrone2020-CC'

In [10]:
#now generate the ShanghaiA's ground truth
#  part_A_train = os.path.join(root,'part_A_final/train_data','images')
 # part_A_test = os.path.join(root,'part_A_final/test_data','images')
  #part_B_train = os.path.join(root,'part_B_final/train_data','images')
 # part_B_test = os.path.join(root,'part_B_final/test_data','images')
#visdrone_train=os.path.join(root,'train')
#visdrone_test=os.path.join(root,'test')
#path_sets = [part_A_train,part_A_test]
mat_train=os.path.join(root,'train','ground_truth')
mat_test=os.path.join(root,'test','ground_truth')
mat_path_sets=[mat_train,mat_test]
mat_paths=[]


In [11]:
 #           img_paths = []
  #          for path in path_sets:
   #             for img_path in glob.glob(os.path.join(path, '*.jpg')):
    #                img_paths.append(img_path)
mat_paths=[]
for path in mat_path_sets:
    for mat_path in glob.glob(os.path.join(path, '*.mat')):
      mat_paths.append(mat_path)

In [ ]:
#for img_path in img_paths:
image_paths=[]
for mat_path in mat_paths:
#    print img_path
    print mat_path
    mat=io.loadmat(mat_path)
    image_path_temp=mat_path.replace('.mat','.jpg').replace('ground_truth','sequence').replace('GT_IMG_','').replace('_','/')
    image_paths.append(image_path_temp)
#    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(image_path_temp)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter_density(k)
    with h5py.File(mat_path.replace('.mat','.h5'), 'w') as hf:
            hf['density'] = k

/content/VisDrone2020-CC/train/ground_truth/GT_IMG_00099_00026.mat
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/ground_truth/GT_IMG_00019_00027.mat
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/ground_truth/GT_IMG_00013_00004.mat
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/ground_truth/GT_IMG_00083_00007.mat
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/ground_truth/GT_IMG_00102_00021.mat
(1080, 1920)
generate density...


In [ ]:
#now see a sample from ShanghaiA
plt.imshow(Image.open(image_paths[0]))

In [ ]:
gt_file = h5py.File(mat_paths[0].replace('.mat','.h5'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation

In [ ]:
!cd CSRNet_pytorch
!python train.py /content/CSRNet_pytorch/test.json /content/CSRNet_pytorch/val.json 0 0

In [ ]:
#!zip -r part_B_final.zip /content/part_B_final